In [1]:
import duckdb
import pandas

db_file = "../ingest_data/tin_chi.duckdb"


try:
    with duckdb.connect(database=db_file, read_only=False) as conn:

        print("\n--- Bảng: GiangVien ---")
        giangvien_df = conn.execute("SELECT * FROM GiangVien").df()
        print(giangvien_df)

        # 2. Kiểm tra bảng MonHoc
        print("\n--- Bảng: MonHoc ---")
        monhoc_df = conn.execute("SELECT * FROM MonHoc").df()
        print(monhoc_df)

        # 3. Kiểm tra bảng LopHocPhan
        print("\n--- Bảng: LopHocPhan ---")
        lophocphan_df = conn.execute("SELECT * FROM LopHocPhan").df()
        print(lophocphan_df)

except Exception as e:
    print(f"Đã có lỗi xảy ra: {e}")


--- Bảng: GiangVien ---
    MaGV           TenGV                 Khoa
0  GV001   Nguyễn Văn An  Công nghệ thông tin
1  GV002   Trần Thị Bích  Công nghệ thông tin
2  GV003     Lê Minh Hải              Kinh tế
3  GV004  Phạm Thị Duyên  Công nghệ thông tin
4  GV005  Hoàng Văn Tuấn            Ngoại ngữ

--- Bảng: MonHoc ---
    MaMH                      TenMH  SoTinChi
0  MH001           Lập trình Python         3
1  MH002              Cơ sở dữ liệu         3
2  MH003              Kinh tế vĩ mô         2
3  MH004        Tiếng Anh giao tiếp         4
4  MH005  Nhập môn Trí tuệ nhân tạo         3

--- Bảng: LopHocPhan ---
  MaLopHP   MaMH   MaGV BuoiHoc          HocKy
0  LHP001  MH001  GV001    Sáng  HK1_2025-2026
1  LHP002  MH001  GV002   Chiều  HK1_2025-2026
2  LHP003  MH002  GV001   Chiều  HK1_2025-2026
3  LHP004  MH003  GV003    Sáng  HK2_2025-2026
4  LHP005  MH004  GV005     Tối  HK1_2025-2026
5  LHP006  MH005  GV002    Sáng  HK2_2025-2026
6  LHP007  MH002  GV004   Chiều  HK2_2025-2026

In [2]:
import re
import requests
from vanna.base import VannaBase


class CustomGroq(VannaBase):
    """
    Lớp tùy chỉnh để kết nối Vanna với Groq Cloud API.
    """

    def __init__(self, config=None):
        if config is None or "api_key" not in config:
            raise ValueError("Config phải chứa 'api_key' cho Groq.")

        self.api_key = config["api_key"]
        self.model = config.get("model", "llama3-70b-8192")  # Đặt model mặc định
        self.temperature = config.get("temperature", 0.7)
        self.base_url = "https://api.groq.com/openai/v1"

    def system_message(self, message: str) -> dict:
        return {"role": "system", "content": message}

    def user_message(self, message: str) -> dict:
        return {"role": "user", "content": message}

    def assistant_message(self, message: str) -> dict:
        return {"role": "assistant", "content": message}

    def submit_prompt(self, prompt, **kwargs) -> str:
        url = f"{self.base_url}/chat/completions"

        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json",
        }

        data = {
            "model": self.model,
            "messages": prompt,
            "temperature": self.temperature,
            "stream": False,
        }

        try:
            response = requests.post(url, headers=headers, json=data)
            # Thêm dòng này để báo lỗi nếu request thất bại (ví dụ: 401, 404, 500)
            response.raise_for_status()

            response_dict = response.json()
            return response_dict["choices"][0]["message"]["content"]

        except requests.exceptions.RequestException as e:
            # Xử lý lỗi một cách rõ ràng
            print(f"Lỗi khi gọi API Groq: {e}")
            if response is not None:
                print(f"Response body: {response.text}")
            return "SELECT 'API_CALL_ERROR' as error;"  # Trả về một SQL lỗi để không làm sập chương trình

    def extract_sql_query(self, text: str) -> str:
        # Giữ nguyên logic trích xuất SQL từ class Vllm
        pattern = re.compile(r"select.*?(?:;|```|$)", re.IGNORECASE | re.DOTALL)
        match = pattern.search(text)
        if match:
            return match.group(0).replace("```", "").strip()
        return text

    def generate_sql(self, question: str, **kwargs) -> str:
            # Gọi phương thức của lớp cha để lấy kết quả thô từ LLM.
            # VannaBase sẽ tự động lo việc tạo prompt và gọi self.submit_prompt
            sql_from_llm = super().generate_sql(question, **kwargs)

            # Bây giờ mới xử lý kết quả trả về
            sql = sql_from_llm.replace("\\_", "_").replace("\\", "")
            return self.extract_sql_query(sql)

    def generate_answer(self, question: str, **kwargs) -> str:
        sql = self.generate_sql(question=question, **kwargs)
    
        df = self.run_sql(sql=sql)

        if df is None or df.empty:
            return "Xin lỗi, tôi không tìm thấy dữ liệu nào phù hợp với câu hỏi của bạn."

        prompt = [
            self.system_message(
                "Bạn là một trợ lý AI. Người dùng đã hỏi một câu hỏi và bạn đã có dữ liệu trả về từ database. "
                "Hãy trả lời câu hỏi của người dùng một cách tự nhiên dựa trên dữ liệu được cung cấp.\n"
                f"Câu hỏi của người dùng là: {question}\n\n"
                f"Dữ liệu từ database là:\n{df.to_string()}"
            )
        ]

        return self.submit_prompt(prompt)


In [ ]:
import vanna as vn
from vanna.openai import OpenAI_Chat
from vanna.chromadb import ChromaDB_VectorStore
# --- BƯỚC 1: THIẾT LẬP VÀ KẾT NỐI ---

# 1.1. Thiết lập Lớp Vanna tùy chỉnh
# Lớp này cho Vanna biết sẽ dùng OpenAI để xử lý ngôn ngữ và ChromaDB để lưu trữ dữ liệu training
class MyVanna(ChromaDB_VectorStore, CustomGroq):
    def __init__(self, config=None):
        ChromaDB_VectorStore.__init__(self, config=config)
        CustomGroq.__init__(self, config=config)

config = {
    "api_key": "",
    "model": "llama-3.3-70b-versatile"
}

vn = MyVanna(config=config)

db_path = '../ingest_data/tin_chi.duckdb' 
vn.connect_to_duckdb(url=db_path)

print(f"Đã kết nối thành công tới database: {db_path}")

# 1. Huấn luyện về cấu trúc (DDL)
vn.train(ddl="""
    CREATE TABLE GiangVien (
        MaGV VARCHAR(10) PRIMARY KEY, TenGV VARCHAR(100) NOT NULL, Khoa VARCHAR(100)
    );
""")
vn.train(ddl="""
    CREATE TABLE MonHoc (
        MaMH VARCHAR(10) PRIMARY KEY, TenMH VARCHAR(150) NOT NULL, SoTinChi INT
    );
""")
vn.train(ddl="""
    CREATE TABLE LopHocPhan (
        MaLopHP VARCHAR(15) PRIMARY KEY, MaMH VARCHAR(10) NOT NULL, MaGV VARCHAR(10) NOT NULL, 
        BuoiHoc VARCHAR(10), HocKy VARCHAR(20)
    );
""")

# 2. Huấn luyện về thuật ngữ và quy tắc (Documentation)
vn.train(documentation="Buổi sáng là các lớp học diễn ra vào buổi sáng.")
vn.train(documentation="Buổi chiều là các lớp học diễn ra vào buổi chiều.")
vn.train(documentation="Buổi tối là các lớp học diễn ra vào buổi tối.")
vn.train(documentation="Học kỳ có định dạng 'HK{số}_{năm học}', ví dụ: 'HK1_2025-2026'.")
vn.train(documentation="Khi người dùng hỏi tên giảng viên không đầy đủ, hãy tìm kiếm gần đúng. Ví dụ 'An' có thể là 'Nguyễn Văn An'.")


# 3. Huấn luyện các kịch bản truy vấn (Question-SQL Pairs)

print("  - Kịch bản 1: Lọc đơn giản")
vn.train(
    question="Gợi ý các lớp học vào buổi sáng",
    sql="SELECT gv.TenGV, mh.TenMH, lhp.BuoiHoc FROM LopHocPhan AS lhp JOIN GiangVien AS gv ON lhp.MaGV = gv.MaGV JOIN MonHoc AS mh ON lhp.MaMH = mh.MaMH WHERE lhp.BuoiHoc = 'Sáng'"
)
vn.train(
    question="Ai dạy môn Lập trình Python?",
    sql="SELECT gv.TenGV FROM LopHocPhan AS lhp JOIN GiangVien AS gv ON lhp.MaGV = gv.MaGV JOIN MonHoc AS mh ON lhp.MaMH = mh.MaMH WHERE mh.TenMH = 'Lập trình Python'"
)

print("  - Kịch bản 2: Lọc kết hợp nhiều điều kiện")
vn.train(
    question="Tôi muốn học môn Lập trình Python của cô Bích",
    sql="SELECT gv.TenGV, mh.TenMH, lhp.BuoiHoc FROM LopHocPhan AS lhp JOIN GiangVien AS gv ON lhp.MaGV = gv.MaGV JOIN MonHoc AS mh ON lhp.MaMH = mh.MaMH WHERE gv.TenGV = 'Trần Thị Bích' AND mh.TenMH = 'Lập trình Python'"
)
vn.train(
    question="Có lớp nào của khoa Công nghệ thông tin vào buổi chiều không?",
    sql="SELECT mh.TenMH, gv.TenGV FROM LopHocPhan lhp JOIN MonHoc mh ON lhp.MaMH = mh.MaMH JOIN GiangVien gv ON lhp.MaGV = gv.MaGV WHERE gv.Khoa = 'Công nghệ thông tin' AND lhp.BuoiHoc = 'Chiều'"
)

print("  - Kịch bản 3: Đếm và tổng hợp (Aggregation)")
vn.train(
    question="Khoa Công nghệ thông tin có bao nhiêu giảng viên?",
    sql="SELECT COUNT(MaGV) FROM GiangVien WHERE Khoa = 'Công nghệ thông tin'"
)
vn.train(
    question="Thầy Nguyễn Văn An dạy bao nhiêu môn?",
    sql="SELECT COUNT(DISTINCT lhp.MaMH) FROM LopHocPhan AS lhp JOIN GiangVien AS gv ON lhp.MaGV = gv.MaGV WHERE gv.TenGV = 'Nguyễn Văn An'"
)

print("  - Kịch bản 4: Xử lý tên không đầy đủ (Fuzzy Matching)")
vn.train(
    question="Gợi ý các môn thầy An dạy",
    sql="SELECT DISTINCT mh.TenMH FROM LopHocPhan AS lhp JOIN GiangVien AS gv ON lhp.MaGV = gv.MaGV JOIN MonHoc AS mh ON lhp.MaMH = mh.MaMH WHERE gv.TenGV LIKE '%An%'"
)
vn.train(
    question="Tôi thích học buổi chiều, có lớp nào của cô Duyên không?",
    sql="SELECT mh.TenMH, gv.TenGV FROM LopHocPhan AS lhp JOIN GiangVien AS gv ON lhp.MaGV = gv.MaGV JOIN MonHoc AS mh ON lhp.MaMH = mh.MaMH WHERE gv.TenGV LIKE '%Duyên%' AND lhp.BuoiHoc = 'Chiều'"
)

print("  - Kịch bản 5: Truy vấn theo học kỳ")
vn.train(
    question="Các môn trong học kỳ 2 năm học 2025-2026",
    sql="SELECT mh.TenMH, gv.TenGV FROM LopHocPhan lhp JOIN MonHoc mh ON lhp.MaMH = mh.MaMH JOIN GiangVien gv ON lhp.MaGV = gv.MaGV WHERE lhp.HocKy = 'HK2_2025-2026'"
)

print("✅ Huấn luyện toàn diện hoàn tất!")




True
Đã kết nối thành công tới database: ../ingest_data/tin_chi.duckdb
Adding ddl: 
    CREATE TABLE GiangVien (
        MaGV VARCHAR(10) PRIMARY KEY, TenGV VARCHAR(100) NOT NULL, Khoa VARCHAR(100)
    );

Adding ddl: 
    CREATE TABLE MonHoc (
        MaMH VARCHAR(10) PRIMARY KEY, TenMH VARCHAR(150) NOT NULL, SoTinChi INT
    );

Adding ddl: 
    CREATE TABLE LopHocPhan (
        MaLopHP VARCHAR(15) PRIMARY KEY, MaMH VARCHAR(10) NOT NULL, MaGV VARCHAR(10) NOT NULL, 
        BuoiHoc VARCHAR(10), HocKy VARCHAR(20)
    );

Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
Adding documentation....
  - Kịch bản 1: Lọc đơn giản
  - Kịch bản 2: Lọc kết hợp nhiều điều kiện
  - Kịch bản 3: Đếm và tổng hợp (Aggregation)
  - Kịch bản 4: Xử lý tên không đầy đủ (Fuzzy Matching)
  - Kịch bản 5: Truy vấn theo học kỳ
✅ Huấn luyện toàn diện hoàn tất!


In [4]:
# --- Bây giờ bạn có thể bắt đầu hỏi ---
print("\nBây giờ bạn có thể bắt đầu hỏi Vanna:")

# Danh sách các câu hỏi để test
test_questions = [
    "Ai dạy môn Kinh tế vĩ mô?",
    "Thầy An có dạy lớp nào vào buổi chiều không?",
    "Cô Bích dạy tổng cộng bao nhiêu môn?",
    "Tìm các lớp của cô Duyên.",
    "Ngoài thầy An, còn ai dạy môn Cơ sở dữ liệu không?"
]

# Vòng lặp qua từng câu hỏi, lấy câu trả lời và in ra
for question in test_questions:
    print(f"❓ User: {question}")
    answer = vn.generate_answer(question=question)
    print(f"🤖 Vanna: {answer}\n")


Bây giờ bạn có thể bắt đầu hỏi Vanna:
❓ User: Ai dạy môn Kinh tế vĩ mô?
SQL Prompt: [{'role': 'system', 'content': "You are a DuckDB SQL expert. Please help to generate a SQL query to answer the question. Your response should ONLY be based on the given context and follow the response guidelines and format instructions. \n===Tables \n\n    CREATE TABLE GiangVien (\n        MaGV VARCHAR(10) PRIMARY KEY, TenGV VARCHAR(100) NOT NULL, Khoa VARCHAR(100)\n    );\n\n\n\n    CREATE TABLE MonHoc (\n        MaMH VARCHAR(10) PRIMARY KEY, TenMH VARCHAR(150) NOT NULL, SoTinChi INT\n    );\n\n\n\n    CREATE TABLE LopHocPhan (\n        MaLopHP VARCHAR(15) PRIMARY KEY, MaMH VARCHAR(10) NOT NULL, MaGV VARCHAR(10) NOT NULL, \n        BuoiHoc VARCHAR(10), HocKy VARCHAR(20)\n    );\n\n\n\n===Additional Context \n\nKhi người dùng hỏi tên giảng viên không đầy đủ, hãy tìm kiếm gần đúng. Ví dụ 'An' có thể là 'Nguyễn Văn An'.\n\nBuổi tối là các lớp học diễn ra vào buổi tối.\n\nBuổi sáng là các lớp học diễn ra 